In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [2]:
spark1 = SparkSession.builder.appName('test').getOrCreate()

In [3]:
#Create a Spark Session
SpSession = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("test") \
    .config("spark.executor.memory", "0.1g") \
    .config("spark.cores.max","2") \
    .config("spark.sql.warehouse.dir", "/home/sushant/Projects/Spark_Project/temp")\
    .getOrCreate()
    
#Get the Spark Context from Spark Session    
SpContext = SpSession.sparkContext

#Test Spark
testData = SpContext.parallelize([3,6,4,2])
testData.count()
#check http://localhost:4040 to see if Spark is running


4

In [4]:
#Spark has a lazy execution
tweetsRDD = SpContext.textFile('movietweets.csv')
display(tweetsRDD.take(5))
ucrRDD = tweetsRDD.map(lambda x: x.upper())
display(ucrRDD.take(5))
tweetsRDD.count()
tweetsRDD.collect() # only for small dataset
tweetsRDD.first()
autoData = SpContext.textFile("auto-data.csv")
autoData.cache()
print autoData.take(5)
autoData.count()
#for line in autoData.collect():
    #print line
# Save loaded data to normal file
autoDataFile = open("auto-Data-save.csv","w")
autoDataFile.write('\n'.join(autoData.collect()))
autoDataFile.close()

[u'positive,The Da Vinci Code book is just awesome.',
 u'positive,i liked the Da Vinci Code a lot.',
 u'positive,i liked the Da Vinci Code a lot.',
 u"positive,I liked the Da Vinci Code but it ultimatly didn't seem to hold it's own.",
 u"positive,that's not even an exaggeration ) and at midnight we went to Wal-Mart to buy the Da Vinci Code"]

[u'POSITIVE,THE DA VINCI CODE BOOK IS JUST AWESOME.',
 u'POSITIVE,I LIKED THE DA VINCI CODE A LOT.',
 u'POSITIVE,I LIKED THE DA VINCI CODE A LOT.',
 u"POSITIVE,I LIKED THE DA VINCI CODE BUT IT ULTIMATLY DIDN'T SEEM TO HOLD IT'S OWN.",
 u"POSITIVE,THAT'S NOT EVEN AN EXAGGERATION ) AND AT MIDNIGHT WE WENT TO WAL-MART TO BUY THE DA VINCI CODE"]

[u'MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE', u'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118', u'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151', u'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195', u'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348']


In [5]:
#Transformation
tsvData = autoData.map(lambda x: x.replace(",","\t"))
tsvData.take(5)
toyotaData = autoData.filter(lambda x: "toyota" in x)
toyotaData.count()
words = toyotaData.flatMap(lambda line : line.split(','))
words.take(20)
words.count()
for x in testData.distinct().collect():
    print x
word1 = SpContext.parallelize(['hey','where','are','you'])
word2 = SpContext.parallelize(['I', 'hey','am','coming'])
for intersects in word1.union(word2).distinct().collect():
    print intersects
for intersects in word1.intersection(word2).distinct().collect():
    print intersects

#pass a function to be worked on whole data
def cleansingRDD(autostr):
    if isinstance(autostr,int):
        return autostr
    attlist = autostr.split(",")
    attlist[5] = attlist[5].upper()
    return ",".join(attlist)
    
cleanedRDD = autoData.map(cleansingRDD)
cleanedRDD.collect()

2
3
4
6
I
am
you
where
coming
hey
are
hey


[u'MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 u'subaru,gas,std,two,hatchback,FWD,four,69,4900,31,36,5118',
 u'chevrolet,gas,std,two,hatchback,FWD,three,48,5100,47,53,5151',
 u'mazda,gas,std,two,hatchback,FWD,four,68,5000,30,31,5195',
 u'toyota,gas,std,two,hatchback,FWD,four,62,4800,35,39,5348',
 u'mitsubishi,gas,std,two,hatchback,FWD,four,68,5500,37,41,5389',
 u'honda,gas,std,two,hatchback,FWD,four,60,5500,38,42,5399',
 u'nissan,gas,std,two,sedan,FWD,four,69,5200,31,37,5499',
 u'dodge,gas,std,two,hatchback,FWD,four,68,5500,37,41,5572',
 u'plymouth,gas,std,two,hatchback,FWD,four,68,5500,37,41,5572',
 u'mazda,gas,std,two,hatchback,FWD,four,68,5000,31,38,6095',
 u'mitsubishi,gas,std,two,hatchback,FWD,four,68,5500,31,38,6189',
 u'dodge,gas,std,four,hatchback,FWD,four,68,5500,31,38,6229',
 u'plymouth,gas,std,four,hatchback,FWD,four,68,5500,31,38,6229',
 u'chevrolet,gas,std,two,hatchback,FWD,four,70,5400,38,43,6295',
 u'toyota,gas,std,two,hatchback,FWD,f

In [6]:
#reduce operations on RDD
testData.collect()
testData.reduce(lambda x, y: x+y)
autoData.reduce(lambda x,y : y if len(x)> len(y) else x)

#Find average MPG for all cars

def getMPG(autostr):
    if isinstance(autostr, int):
        return autostr
    attlist = autostr.split(",")
    if attlist[9].isdigit():
        return int(attlist[9])
    else:
        return 0
autoData.reduce(lambda x,y : getMPG(x)+getMPG(y))/(autoData.count()-1) # -1 to reduce the header row

25

In [7]:
#Keyvalue pair
kpairData = autoData.map(lambda x:(x.split(',')[0],x.split(',')[7]))
kpairData.take(5)
kpairData.keys().collect()
header = kpairData.first()
kpairData = kpairData.filter(lambda line: line !=header)

# Find average mpg for every brand
addOne = kpairData.mapValues(lambda x : (x,1))
brandValues = addOne.reduceByKey(lambda x,y:(int(x[0])+int(y[0]),x[1]+y[1]))
averageBrand = brandValues.mapValues(lambda x: int(x[0])/int(x[1])).collect()

In [8]:
# Accumulator, broadcast, persistance of variables

sedanCount = SpContext.accumulator(0)
hatchbackCount = SpContext.accumulator(0)

sedanText = SpContext.broadcast("sedan")
hatchbackText = SpContext.broadcast("hatchback")

def splitlines(line):
    global sedanCount
    global hatchbackCount
    
    if sedanText.value in line:
        sedanCount += 1
    elif hatchbackText.value in line:
        hatchbackCount += 1
        
    return line.split(',')

autoTypeCount = autoData.map(splitlines)
autoTypeCount.count()
print hatchbackCount, sedanCount

testData.getNumPartitions()
#partition defination
newRDD = SpContext.parallelize([1,2,3,4,5],3)
newRDD.cache()
newRDD.collect()
newRDD.getNumPartitions()

67 92


3

In [9]:
#Spark SQL interface
empDf = SpSession.read.json("customerData.json")
empDf.show()
empDf.printSchema()
#Queries
empDf.select("name").show()
empDf.filter(empDf["age"]==40).show()
empDf.groupBy("gender").count().show()
empDf.groupBy("deptid").agg({"salary":"avg","age":"max"}).show()
deptList = [{'name':'Sales', 'id':'100'},{'name':'Engineering', 'id':'200'}]
deptDf = SpSession.createDataFrame(deptList)
deptDf.show()
empDf.join(deptDf,empDf.deptid == deptDf.id).show()

#cascading operations
empDf.filter(empDf["age"] >30).join(deptDf, empDf.deptid == deptDf.id).groupBy("deptid").agg({"salary":"avg","age":"max"}).show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+

+------+-----+
|

/usr/lib/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+

+---+------+------+-----------------+------+---+-----------+
|age|deptid|gender|             name|salary| id|       name|
+---+------+------+-----------------+------+---+-----------+
| 51|   100|female|    June Rutledge|  4300|100|      Sales|
| 26|   100|  male|  Burks Velasquez|  2700|100|      Sales|
| 32|   100|  male|Benjamin Garrison|  3000|100|      Sales|
| 44|   200|  male|    Nielsen Knapp|  6500|200|Engineering|
| 40|   200|  male|    Holland Drake|  4500|200|Engineering|
+---+------+------+-----------------+------+---+-----------+

+------+-----------+--------+
|deptid|avg(salary)|max(age)|
+------+-----------+--------+
|   200|     5500.0|      44|
|   100|     3650.0|      51|
+------+-----------+--------+



In [10]:
# create a Data Frame from RDD
from pyspark.sql import Row
lines = SpContext.textFile("auto-data.csv")
#remove first line
datalines = lines.filter(lambda x : "FUELTYPE" not in x)
datalines.count()
parts = datalines.map(lambda l: l.split(","))
autoMap = parts.map(lambda x: Row(make= x[0],body=x[4],hp =int(x[7])))
autoDf = SpSession.createDataFrame(autoMap)
autoDf.show()
#direct import for dataframe
autoDf1 = SpSession.read.csv("auto-data.csv",header=True)
autoDf1.show()


+---------+---+----------+
|     body| hp|      make|
+---------+---+----------+
|hatchback| 69|    subaru|
|hatchback| 48| chevrolet|
|hatchback| 68|     mazda|
|hatchback| 62|    toyota|
|hatchback| 68|mitsubishi|
|hatchback| 60|     honda|
|    sedan| 69|    nissan|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 68|     mazda|
|hatchback| 68|mitsubishi|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 70| chevrolet|
|hatchback| 62|    toyota|
|hatchback| 68|     dodge|
|hatchback| 58|     honda|
|hatchback| 62|    toyota|
|hatchback| 76|     honda|
|    sedan| 70| chevrolet|
+---------+---+----------+
only showing top 20 rows

+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|      MAKE|FUELTYPE|ASPIRE|DOORS|     BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|    subaru|     gas|   std|  two|hatchback|  f

In [13]:
#tempTables
autoDf.createOrReplaceTempView("autos")
SpSession.sql("select * from autos where hp>200").show()
#to pandas
empPandas =empDf.toPandas()
for index,row in empPandas.iterrows():
    print row["salary"]

#Working with Database
# download->copy mysql-connector-java-3.0.17-ga-bin.jar  in /usr/lib/spark-2.4.0-bin-hadoop2.7/jars and restart python
#Example follow
'''demoDf = SpSession.read.format("jdbc").options(
          url="jdbc:mysql://localhost:3306/demo",
          driver = "com.mysql.jdbc.Driver",
          dbtable = "demotable",
          user = "admin",
          password = "admin").load()
demoDf.show()'''
          

+-----------+---+-------+
|       body| hp|   make|
+-----------+---+-------+
|    hardtop|207|porsche|
|    hardtop|207|porsche|
|      sedan|262| jaguar|
|convertible|207|porsche|
+-----------+---+-------+

3000
4500
2700
4300
6500


'demoDf = SpSession.read.format("jdbc").options(\n          url="jdbc:mysql://localhost:3306/demo",\n          driver = "com.mysql.jdbc.Driver",\n          dbtable = "demotable",\n          user = "admin",\n          password = "admin").load()\ndemoDf.show()'

In [14]:
#Streaming
from pyspark.streaming import StreamingContext
#start a streaming on following 9000 port and process the streaming RDD 
#Create streaming context with latency of 1


'''streamContext = StreamingContext(SpContext,3)

totalLines=0
lines = streamContext.socketTextStream("localhost", 9000)


#Word count within RDD    
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)
wordCounts.pprint(5)

#Count lines
totalLines=0
linesCount=0
def computeMetrics(rdd):
    global totalLines
    global linesCount
    linesCount=rdd.count()
    totalLines+=linesCount
    print(rdd.collect())
    print("Lines in RDD :", linesCount," Total Lines:",totalLines)

lines.foreachRDD(computeMetrics)

#Compute window metrics
def windowMetrics(rdd):
    print("Window RDD size:", rdd.count())
    
windowedRDD=lines.window(6,3)
windowedRDD.foreachRDD(windowMetrics)

streamContext.start()
streamContext.stop()
print("Overall lines :", totalLines)'''